<a href="https://colab.research.google.com/github/ced-sys/AI---ML/blob/main/Quantum_BERT_LLm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torch accelerate datasets trl peft bitsandbytes
!pip install wandb -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 14.0 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import(
    AutoTokenizer, AutoModelForCausalLM,
    TrainingArguments, Trainer,
    pipeline
)

from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from peft import LoraConfig, get_peft_model, TaskType
import numpy as np
import json
import random
from typing import List, Dict, Tuple, Optional
from dataclasses import dataclass
import re

In [ ]:
import bitsandbytes
print(bitsandbytes.__version__)

0.47.0


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import(
    AutoTokenizer, AutoModelForCausalLM,
    TrainingArguments, Trainer,
    pipeline, BitsAndBytesConfig
)

from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from peft import LoraConfig, get_peft_model, TaskType
import numpy as np
import json
import random
from typing import List, Dict, Tuple, Optional
from dataclasses import dataclass
import re

In [ ]:
class QuantumPhysicsLLM:
  def __init__(self, model_name="microsoft/DialoGPT-small", use_lora=True):
    self.device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {self.device}")

    self.tokenizer=AutoTokenizer.from_pretrained(model_name)
    if self.tokenizer.pad_token is None:
      self.tokenizer.pad_token=self.tokenizer.eos_token

    if torch.cuda.is_available():
      quantization_config=BitsAndBytesConfig(
          load_in_4bit=True,
          bnb_4bit_quant_type="nf4",
          bnb_4bit_use_double_quant=True,
          bnb_4bit_compute_dtype=torch.float16
      )
    else:
      quantization_config=None

    self.model=AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=quantization_config,
        device_map="auto" if torch.cuda.is_available() else torch.float32
    )

    if use_lora:
      lora_config=LoraConfig(
          task_type=TaskType.CAUSAL_LM,
          inference_model=False,
          r=16,
          lora_alpha=32,
          lora_dropout=0.1,
          target_modules=["c_attn", "c_proj"]
      )

      self.model=get_peft_model(self.model, lora_config)
      print("LoRA applied for efficient training")

    self.value_model=None
    self.ppo_trainer=None

    self.quantum_knowledge=self._build_knowledge_base()
    self.teaching_strategies=self._buil_teaching_strategies()

  def _build_knowledge_base(self)-> Dict[str, Dict]:
    return{
        "superposition":{
            "simple":"A quantum particle can be in multipe states at once, like a coin spinning in the air",
            "detailed":"Superposition is a fundamental quantum principle where a system exists in all possible states simultaneously until measurement collapses it to a single state.",
            "analogy":"Imagine a magic coin that's both heads and tails while spinning- only when it lands does it 'choose' one side.",
            "math": "\psi = \alpha|0\rangle + \beta |0\rangle where |\alpha|^2 + |\beta|^2=1",
            "examples":["Schrondinger's cat", "Double-slit experiment", "Quantum bits in computers"]
        },
        "entanglement":{
            "simple": "Two particles become magically connected - measure one, instantly affects the other!",
            "detailed":"Quantum entanglement creates correlations between particles such that the quantum state of each particle cannot be described independently.",
            "analogy":"Like having two magical dice that always show opposite numbers, even when thrown on different planets.",
            "math":"Bell state: |\psi\rangle=(|00\rangle+ |11\rangle)/ sqrt",
            "examples":["EPR paradox", "Quantum teleportation", "Quantum cryptography"]
        },
        "uncertainty":{
            "simple":"You can't know exactly where a particle is and how fast it's moving at the same time.",
            "detailed": "Heisenberg's uncertainity principle states that certain pairs of properties (like position and momentum) canot be simltaneously measured with arbitrary precision.",
            "analogy":"Like trying to photograph a race car- you can capture where it is or how fast it's going, but not both perfectly",
            "math":"\deltax x \deltap >= \hat h /2".
            "examples":["Electron microscopy limts", "Quantum tunneling", "Zero-poin energy"]
        },
        "wave-function":{
            "simple":"A mathematical wave that describes all possible states of a quantum system.",
            "detailed":"The wave function contains complete information about a quantum system and evolves according to the Schrondinger's equation.",
            "analogy": "Like a weather map showing probability of rain everywhere- until you go outside and check!",
            "math": "i \hat h \delta \psi/ \delta t=\hat H \psi (Schrondinger's equation)"
        }
    }


  def _build_teaching_strategies(self)-> Dict[str, callable]:
    return{
        "beginner":self._teach_beginner,
        "intermediate":self._teach_intermediate,
        "advanced":self._teach_advanced,
        "visual":self._teach_visual,
        "mathematical":self._teach_mathematical

    }